# MACHINE LEARNING | COVID-19
#### Projeto utilizando os dados do site: [COVID-19](https://covid.saude.gov.br/)
---------------------------------------------------------------------
##### Análise de dados sobre a curva de evolução do Covid-19 durante o ano de 2022 e predição de novos casos até janeiro de 2023.

In [50]:
#Importando as bibliotecas necessárias para criação do projeto:

import re
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go
from datetime import datetime

In [51]:
#Importando os datasets que serão utilizados e concatenando-os:

df1 = pd.read_csv("PainelCovid19Brasil2022-Pt1.csv", on_bad_lines = 'skip', sep = ';')
df2 = pd.read_csv("PainelCovid19Brasil2022-Pt2.csv", on_bad_lines = 'skip', sep = ';')

df = pd.concat([df1, df2])

In [60]:
#Verificando se há pelo menos um caso confirmado e se é no Brasil:

brasil = df.loc[(df.regiao == 'Brasil') & (df.casosAcumulado > 0)]

In [61]:
#Excluindo colunas desnecessárias e alterando o nome:
brasil = brasil.drop(columns = ['estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude', 'nomeRegiaoSaude', 'interior/metropolitana', 'emAcompanhamentoNovos', 'casosNovos', 'obitosNovos'])
brasil = brasil.rename(columns = {"regiao":"pais", "semanaEpi":"semana", "populacaoTCU2019":"populacao", "casosNovos":"novosCasos", "casosAcumulado":"casos", "obitosAcumulado":"obitos", "Recuperadosnovos":"recuperados"})
brasil.head()

,pais,data,semana,populacao,casos,obitos,recuperados
0,Brasil,2022-01-01,52,210147125.0,22291507.0,619105,21581668.0
1,Brasil,2022-01-02,1,210147125.0,22293228.0,619133,21581717.0
2,Brasil,2022-01-03,1,210147125.0,22305078.0,619209,21591847.0
3,Brasil,2022-01-04,1,210147125.0,22323837.0,619384,21603954.0
4,Brasil,2022-01-05,1,210147125.0,22351104.0,619513,21615473.0


In [62]:
#Visualizando os casos confirmados no Brasil durante o ano de 2022:
px.line(brasil, 'data', 'casos', 
        labels={'data':'Período', 'casos':'Número de casos confirmados'},
        title='Casos confirmados no Brasil'
        )

### Novos casos no Brasil



In [63]:
#Função para calcular os novos casos no Brasil por dia:

brasil['casos'] = list(map(
    lambda x: 0 if (x==0) else brasil['casos'].iloc[x] - br['casos'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [64]:
px.line(brasil, x='data', y='casos', title='Novos casos por dia',
       labels={'data': 'Data', 'casos': 'Novos casos'})

### Mortes por Covid no Brasil


In [65]:
#Função para mostrar as mortes por Covid-19 no Brasil

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=brasil.data, y=brasil.obitos, name='Mortes', mode='lines+markers',
              line=dict(color='red'))
)

fig.update_layout(title='Mortes por COVID-19 no Brasil',
                   xaxis_title='Data',
                   yaxis_title='Número de mortes')
fig.show()

# PREDIÇÕES

1.   Análise de série temporal
2.   Criação de um modelo para prever novos casos



In [66]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [67]:
novosCasos = brasil.novosCasos
novosCasos.index = brasil.data

res = seasonal_decompose(novosCasos)

fig, (ax1,ax2,ax3, ax4) = plt.subplots(4, 1,figsize=(10,8))
ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.scatter(novosCasos.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

#Decompondo a série de confirmados
confirmados = brasil.novosCasos
confirmados.index = brasil.data

res2 = seasonal_decompose(confirmados)

fig, (ax1,ax2,ax3, ax4) = plt.subplots(4, 1,figsize=(10,8))
ax1.plot(res2.observed)
ax2.plot(res2.trend)
ax3.plot(res2.seasonal)
ax4.scatter(confirmados.index, res2.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

AttributeError: ignored

In [ ]:
#Predizendo o número de casos confirmados com um AUTO-ARIMA
!pip install pmdarima
from pmdarima.arima import auto_arima
!pip install pmdarima

modelo = auto_arima(confirmados)

pd.date_range('2020-10-01', '2022-10-19')

fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name='Observed'
))

fig.add_trace(go.Scatter(x=confirmados.index, y = modelo.predict_in_sample(), name='Predicted'))
fig.add_trace(go.Scatter(x=pd.date_range('2022-10-20', '2022-11-05'), y=modelo.predict(15), name='Forecast'))
fig.update_layout(title='Previsão de casos confirmados para os próximos 16 dias',
                 yaxis_title='Casos confirmados', xaxis_title='Data')
fig.show()

In [ ]:
#Forecasting com Facebook Prophet
!conda install -c conda-forge fbprophet -y
from fbprophet import Prophet

# preparando os dados
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

# renomeia colunas
train.rename(columns={"data":"ds","novoscasos":"y"},inplace=True)
test.rename(columns={"data":"ds","novoscasos":"y"},inplace=True)
test = test.set_index("ds")
test = test['y']

profeta = Prophet(growth="logistic", changepoints=['2022-09-22', '2022-09-30', '2022-10-10', '2022-10-05', '2022-10-14'])

In [ ]:
#pop = 1000000
pop = 211463256 #https://www.ibge.gov.br/apps/populacao/projecao/box_popclock.php
train['cap'] = pop

# Treina o modelo
profeta.fit(train)

# Construindo previsões para o futuro
future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop
forecast =  profeta.predict(future_dates)

fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=test.index, y=test, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()